# View
n개의 HTTP request -> 1개 view 호출    
`urls.py/urlpatterns` 리스트에 뷰를 매핑함 ,     
    함수의 형태 등 호출 가능한 객체로 구현할 수 있다.
    
웹 클라이언트로부터 HTTP 요청을 받아서 처리

1. 함수 기반 뷰 Function Based View (FBV)
장고 뷰의 기본이므로 이것을 제작할 수 있어야한다.    
2. 클래스 기반 뷰 Class Based View (CBV)
classname.as_view()로 호출 가능    
FBV가 익숙해지면 CBV 하기를 추천

### View 호출 시, 인자는?
parameter : HttpRequest 객체 , URL Captured Values
1. HttpRequest 객체
현재 요청에 대한 모든 내역        
2. URL Captured Values    
```
urlpatterns = [
path('/instagram/', views.post_list),
path('/instagram/1/', views.post_list),
path('/instagram/2/', views.post_list),
]
```
대신에,
```
urlpatterns = [
path('', views.post_list),
path('<int:pk>/', views.post_detail), # post_detail pk 값이 int로 넘어감
re_path(r'(?P<pk>\d+)/$', views.post_detail), # post_detail pk 값이 string으로 넘어감
]
```

In [ ]:
def post_list(request):
    pass
def post_detail(request, pk): # 정규표현식에서 사용한 인자명 그대로 사용해야함
    pass

path를 통한 처리 : 매핑된 Converter(<int:pk> 이부분, 정규표현식이 정의된 클래스임)에 to_python에 맞게 변환된 값이 전달됨

### View 호출에 대한 return value
- HttpResponse 객체 반환해야함!
View를 middleware가 감싸서, request는 middleware를 거친뒤 view로 향한다.
그래서 HttpResponse 객체가 아닌 다른 타입을 반환하면,    
object has no attribute 'get' 미들웨어에서 에러날 수 있다.    
파일처럼 함수를 사용할 수 있다.

In [1]:
def post_detail(request: HttpRequest, pk:int) -> HttpResponse:
    response = HttpResponse()
    response.write("Hello World")
    return response

**request 속성들**    
request.method    
request.META    
request.GET , .POST , .FILES , .body

### FBV example
myapp/views.py

In [2]:
def item_list(request):
    qs = Item.objects.all()
    return render(request, 'shop/item_list.html', {
        'item_list': qs,
    })

myapp/urls.py

In [ ]:
urlpatterns = [
    path('items/', item_list, name='item_list'),
]

### CBV example

In [ ]:
class GenericListView:
    ...
    def __call__(self, request):
        pass

item_list = GenericListView(Item)

urlpatterns = [
    path('items/', item_list, name='item_list'),
]

__call__ 가 구현되어있으면 item_list 을 urlpatterns 에 적어두면 호출된다.

In [ ]:
from django.views.generic import ListView

post_list = ListView.as_view(model=Post)

urlpatterns = [
    path('items/', item_list, name='item_list'),
]

In [ ]:
class ItemListView(ListView):
    model = Item
    
item_list = ItemListView.as_view()

urlpatterns = [
    path('items/', item_list, name='item_list'),
]

### View에서
Excel 파일 다운로드 응답만들기    
- with ... as 사용
- pandas library 사용

이미지 response 만들기
- pillow 사용

# URL Dispatcher와 정규 표현식
- 주소 = 서버의위치/요청/ URL/
- urlpatterns list = 특정 URL pattern은 특정 View가 처리 
- 최상위 URL은 project/settings.py에서 ROOT_URLCONF 모듈로 지정
- 최상위 urlpatterns에서 include를 사용하여 Tree 구조로 url 확장    
- 순차적으로 url을 확인하기 때문에 가장 먼저 적합한 url path가 불러와진다.
- 매칭된게 없으면 404

### urlpatterns example
기본 제공되는 Path Converters를 이용해서 정규표현식을 편리하게 사용할 수 있다.

**정규 표현식**
- 문자열의 패턴, 규칙을 정의하는 방법
- 장고 URL Dispatcher에서 정규표현식을 통한 URL 매칭을 한다.
- 다양한 정규 표현식 패턴, 반복횟수 지정 문법보고 작성하면 된다.

### Custom path Convert
`path('appname/<convertname:parametername>/', views.archives.year)`

In [ ]:
class YearConvert:
    regex = r"20\d{2}"
    
    def to_python(self, value): # 정수로 변환
        return int(value)
    def to_url(self, value): # 어떤 값을 문자열 reverse 다시 변환 
        return "%04d" % value

register_converter(YearConvert, 'year')

urlpatterns = [
    ...
    path('archives/<year:year>/', views.archives.year),
]

1. 앱 생성
2. 앱 이름/urls.py 파일 생성
3. 프로젝트/urls.py에 include 적용
4. 프로젝트/settings.py INSTALLED_APPS에 앱 이름 등록

